# use_keras

Build a neural network with Keras to try to identify drawn digits.

In [68]:
import tkinter as tk
import numpy as np

# Geometry constants.
WINDOW_WID = 160
WINDOW_HGT = 380
FRAME1_WID = 150
PADX = 5
NETWORK_CANVAS_WID = WINDOW_WID - FRAME1_WID - 4 * PADX
NETWORK_CANVAS_HGT = WINDOW_HGT - 2 * PADX
NUM_ROWS = 8
NUM_COLS = 6
CELL_WID = 20
CELL_HGT = CELL_WID
MARGIN = 5

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('digit_identification')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Initially we have nothing to draw.
        self.polyline = None
        self.points = []

        # Build the UI.
        self.build_ui()

        # Build and train the neural network.
        self.make_trained_model()

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    # Redraw the digit-drawing area.
    def redraw(self):
        # Remove old polyline.
        self.grid_canvas.delete(self.polyline)
        self.polyline = None

        # Draw current points.
        if len(self.points) > 1:
            self.polyline = self.grid_canvas.create_line(self.points, fill='black')

    def build_ui(self):
        # Make controls to define the network.
        frame1 = tk.Frame(self.window, width=FRAME1_WID)
        frame1.pack(side=tk.LEFT, expand=False, fill=tk.Y, padx=PADX)
        frame1.pack_propagate(False)

        frame2 = tk.Frame(self.window)
        frame2.pack(side=tk.LEFT, expand=True, fill=tk.BOTH, padx=PADX)

        # Make the drawing canvas.
        canvas_wid = NUM_COLS * CELL_WID + 1
        canvas_hgt = NUM_ROWS * CELL_HGT + 1
        self.grid_canvas = tk.Canvas(frame1, bg='white',
            borderwidth=0, highlightthickness=0, relief=tk.SUNKEN, width=canvas_wid, height=canvas_hgt)
        self.grid_canvas.pack(side=tk.TOP, pady=5)
        self.grid_canvas.bind('<Button-1>', self.start_draw)
        self.grid_canvas.bind('<ButtonRelease-1>', self.end_draw)

        # Make grid lines.
        for r in range(NUM_ROWS + 1):
            self.grid_canvas.create_line(0, r * CELL_HGT, canvas_wid, r * CELL_HGT, fill='lime')
        for c in range(NUM_COLS + 1):
            self.grid_canvas.create_line(c * CELL_WID, 0, c * CELL_WID, canvas_hgt, fill='lime')

        # Make a big label to display results from the user drawing.
        self.user_result_var = tk.StringVar()
        self.user_result_label = tk.Label(frame1, font=('Calibri 80 normal'),
            textvariable=self.user_result_var)
        self.user_result_label.pack(side=tk.TOP)

        # Make a label to display accuracy.
        self.accuracy_var = tk.StringVar()
        self.accuracy_label = tk.Label(frame1, textvariable=self.accuracy_var)
        self.accuracy_label.pack(side=tk.TOP)

        # Make a label to display accuracy.
        self.val_accuracy_var = tk.StringVar()
        self.val_accuracy_label = tk.Label(frame1, textvariable=self.val_accuracy_var)
        self.val_accuracy_label.pack(side=tk.TOP)

    def start_draw(self, event):
        # Clear any previous result.
        self.user_result_var.set('')

        # Remove any previous drawing.
        self.points = []
        self.redraw()

        self.grid_canvas.bind('<B1-Motion>', self.save_point)

    def end_draw(self, event):
        self.grid_canvas.unbind('<B1-Motion>')

        # Evaluate the polyline.
        self.evaluate_polyline()

    def save_point(self, event):
        self.points.append((event.x, event.y))
        self.redraw()

    def kill_callback(self):
        self.window.destroy()

    # See which digit is most likely.
    def evaluate_polyline(self):
        # Set VERBOSE = True to see diagnostic output.
        VERBOSE = False

        # Convert the polyline into a list of cells touched.
        inputs = [ self.polyline_to_0_1_list() ]
        if VERBOSE:
            print(f'inputs: {inputs}')

        # Run the neural network.
        predictions = self.model.predict(inputs)
        if VERBOSE:
            print(f'predictions: {predictions}')

        # Find the index of the largest output value.
        best_i = 0
        best_value = predictions[0][0]
        for i in range(len(predictions[0])):
            if predictions[0][i] > best_value:
                best_i = i
                best_value = predictions[0][i]
        if VERBOSE:
            print(f'best_value: {best_value}')
            print(f'best_i: {best_i}')

        # Display the best result.
        self.user_result_var.set(f'{best_i}')

    # Convert the polyline into a list of 0s and 1s showing which cells were touched.
    def polyline_to_0_1_list(self):
        # Make a list holding all 0s.
        touched_list = [0 for i in range(NUM_ROWS * NUM_COLS)]

        # Mark the touched cells.
        for point in self.points:
            r = int(point[1] / CELL_HGT)
            c = int(point[0] / CELL_WID)
            if r >= 0 and r < NUM_ROWS and c >= 0 and c < NUM_COLS:
                index = r * NUM_COLS + c
                touched_list[index] = 1

        # Return the list.
        return touched_list

    # For the general approach, see:
    #     https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
    # Note that the input layer is usually not counted in the number of layers so when
    # the article creates a "three-layer network" that's an input layer (not counted),
    # two hidden layers, and an output layer. We're only making one hidden layer.
    # Save the trained model in self.model and display the last accurracy value.
    def make_trained_model(self):
        # Set VERBOSE = True to see diagnostic output.
        VERBOSE = False
        SHOW_PLOTS = True
    
        #################
        # Load the data #
        #################
        import pandas as pd
    
        # Read the data into a dataframe.
        df = pd.read_csv('digit_data.csv')
    
        # Print the shape of the data. It should be 79 rows with 58 columns.
        if VERBOSE:
            print(f'df.shape: {df.shape}')
    
        # Split data into training and validation sets.
        # (This would work better with many more rows.)
        from sklearn.model_selection import train_test_split
        TEST_FRACTION = 0.2
        train_df, test_df = train_test_split(df, test_size=TEST_FRACTION)
        if VERBOSE:
            print(
                f'Using {len(train_df)} training samples '
                f'and {len(test_df)} test samples'
            )
    
        # The input data includes columns 0 through 9.
        # The output data includes columns 10 through 58.
        # Both the inputs and outputs are vectors so they are capitalized.
        # (I.e. there are multiple inputs and outputs for each data sample.)
        # Get the inputs in columns 10 through 58.
        train_X = train_df.iloc[:, 10:59]
        test_X = test_df.iloc[:, 10:59]

        # Get the outputs in columns 0 through 9.
        train_Y = train_df.iloc[:, 0:10]
        test_Y = test_df.iloc[:, 0:10]
    
        if VERBOSE:
            # Print the test inputs and outputs to verify that it makes sense.
            print('test_df:')
            display(test_df)
            print('test_X:')
            display(test_X)
            print('test_Y:')
            display(test_Y)
    
        # Build a network with an input layer, one hidden layer, and an output layer.
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense
        NUM_INPUTS = 48
        NUM_HIDDEN = 10
        NUM_OUTPUTS = 10
        model = Sequential()
    
        # Define the input and hidden layer. (The first layer also defines the input layer.)
        # "Dense" means each neuron is connected to every input.
        # Parameters:
        #     NUM_HIDDEN    This layer contains NUM_HIDDEN neurons.
        #     input_shape   Each neuron's inputs have NUM_INPUTS inputs. The trailing
        #                   comma makes this a list containing the single value NUM_INPUTS.
        #     activation    Use the ReLU activation function. These days, the rectified linear
        #                   unit (ReLU) activation function is in vogue for non-output layers.
        #     use_bias      We want to use bias. (This is the default.)
        model.add(Dense(NUM_HIDDEN, input_shape=(NUM_INPUTS,), activation='relu', use_bias=True))
    
        # Define the output layer.
        # Many models use a sigmoid activation function to map the result to 0 to 1.
        model.add(Dense(NUM_OUTPUTS, activation='sigmoid'))
    
        # Compile the model.
        model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
        if VERBOSE:
            print('Compiled')

        # Train the model.
        NUM_EPOCHS = 400
        BATCH_SIZE = 10
        # Other parameters:
        #    verbose=0       No progress reports
        #    shffle=False    Do not shuffle data before each epoch
        history = model.fit(train_X, train_Y, validation_data=(test_X, test_Y), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)
        if VERBOSE:
            print('Trained')
    
        if SHOW_PLOTS:
            # Plot the loss and accuracy during training.
            from matplotlib import pyplot

            # Draw plots in a new window.
            %matplotlib qt

            # Plot loss.
            pyplot.figure()
            pyplot.title('Loss')
            pyplot.plot(history.history['loss'], label='train')
            pyplot.plot(history.history['val_loss'], label='test')
            pyplot.legend()
            pyplot.show()

            # Plot accuracy.
            pyplot.figure()
            pyplot.title('Accuracy')
            pyplot.plot(history.history['accuracy'], label='train')
            pyplot.plot(history.history['val_accuracy'], label='validate')
            pyplot.legend()
            pyplot.show()
    
        # Display the final accurracy and val_accuracy values.
        accuracy = history.history.get('accuracy')[-1]
        self.accuracy_var.set(f'Accuracy: {accuracy:.2%}')
        val_accuracy = history.history.get('val_accuracy')[-1]
        self.val_accuracy_var.set(f'Val Accuracy: {val_accuracy:.2%}')
    
        # Save the trained model.
        self.model = model

In [69]:
App()

Epoch 1/400
7/7 [==============================] - 0s 26ms/step - loss: 0.2523 - accuracy: 0.0794 - val_loss: 0.2504 - val_accuracy: 0.0625
Epoch 2/400
7/7 [==============================] - 0s 6ms/step - loss: 0.2360 - accuracy: 0.0794 - val_loss: 0.2369 - val_accuracy: 0.0625
Epoch 3/400
7/7 [==============================] - 0s 6ms/step - loss: 0.2210 - accuracy: 0.0952 - val_loss: 0.2248 - val_accuracy: 0.0625
Epoch 4/400
7/7 [==============================] - 0s 6ms/step - loss: 0.2078 - accuracy: 0.1270 - val_loss: 0.2128 - val_accuracy: 0.0625
Epoch 5/400
7/7 [==============================] - 0s 6ms/step - loss: 0.1953 - accuracy: 0.1270 - val_loss: 0.2009 - val_accuracy: 0.0625
Epoch 6/400
7/7 [==============================] - 0s 6ms/step - loss: 0.1837 - accuracy: 0.1270 - val_loss: 0.1897 - val_accuracy: 0.1250
Epoch 7/400
7/7 [==============================] - 0s 6ms/step - loss: 0.1733 - accuracy: 0.1111 - val_loss: 0.1795 - val_accuracy: 0.0625
Epoch 8/400
7/7 [=========